In [158]:
import numpy
import pandas
import nltk
import sys
import mysecrets
import random

<h1> Get positive and negative words from NRC Emotion Lexicon </h1>

In [2]:
def get_nrc_data(emotion):
    nrc = "/Users/rohankulkarni/nltk_data/NRC-Emotion-Lexicon/OneFilePerEmotion/"+ emotion +"-NRC-Emotion-Lexicon.txt" 

    
    f = open(nrc,'r')
    words = [word.split('\t')[0] for i, word in enumerate(f)]

    return words


In [3]:
pos = get_nrc_data("positive")
neg = get_nrc_data("negative")

<h1> Use Yelp API to get sample reviews </h1>

In [4]:
yelp_client_id = mysecrets.yelp_client_id
yelp_api_key = mysecrets.yelp_api_key

In [5]:
API_HOST = 'https://api.yelp.com' #The API url header
SEARCH_PATH = '/v3/businesses/search' #The path for an API request to find businesses
BUSINESS_PATH = '/v3/businesses/'  # The path to get data for a single business


In [15]:
def get_restaurants(api_key,location,number=15):
    import requests
    
     #Set up the search data dictionary. This contains the search parameters
    search_data = {
    'term': "restaurant",
    'location': location.replace(' ', '+'),
    'limit': number
    }
    
    #Create the API url
    url = API_HOST + SEARCH_PATH
    
    #The API Key data object
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }
    
    #Get the response object and convert from JSON into a python list
    response = requests.request('GET', url, headers=headers, params=search_data).json()
    #print(response)
    
    #Extract the businesses from the response object
    businesses = response.get('businesses')
    
    #Extract the business id and business name for each business into a list
    return_data = [(business['id'],business['name']) for business in businesses]
    return return_data

get_restaurants(yelp_api_key,"Princeton",15)

#Let's choose the restaurant: Tacoria mC0Oir4IMjMSzJGORIW6pw

[('ANG7pXCpE1iPzDSsB-abwg', 'The Meeting House'),
 ('SgXxGCzsf5xhEo5mYpBHEw', 'Agricola Eatery'),
 ('ytvXEa1BdgdMQPOGANLBCQ', 'The Dinky Bar & Kitchen'),
 ('aA3pwxdS76TCPskajGj_Qg', 'Mistral'),
 ('K08kXfrLkM6A7Zixwb5ljQ', 'Roots Ocean Prime'),
 ('3g-OXwOHASK8bCuInOfNKg', 'Blue Point Grill'),
 ('0YkXhcfpFHuYw_33Iq9kgQ', 'The Perch At Peacock Inn'),
 ('LxL23QJGTuiNW_lfE1mn8Q', 'Local Greek'),
 ('Q0hNFisTc8hm5t1sCVHB4A', 'Schouse'),
 ('BZ90gSrU0PdZTpGBaXYMUg', 'Witherspoon Grill'),
 ('4CXO92H2UsgSI4koZNTBQg', 'Salt Creek Grille'),
 ('zdj5iGqtqJmLBL1UFsa5mA', 'Lan Ramen'),
 ('mC0Oir4IMjMSzJGORIW6pw', 'Tacoria'),
 ('wFXwhiQ3Hp2SsJb-iNG4BA', 'Elements'),
 ('rXy_19w4bAoyGImmZ4w23Q', 'Kristines')]

In [13]:
taco = "mC0Oir4IMjMSzJGORIW6pw"

In [17]:
def get_business_review(api_key,business_id):
    import requests
    
    #API path
    business_path = BUSINESS_PATH + business_id+"/reviews"
    
    #API url
    url = API_HOST + business_path

    #API Key data object
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    #Get the response and convert from json into a python object
    response = requests.request('GET', url, headers=headers).json()
    #print(response)
    
    #Yelp gives review snippets of a few reviews, concatenate these into a single string
    review_text = ''
    for review in response['reviews']:
        review_text += review['text']
    return review_text

review = get_business_review(yelp_api_key,taco)
review

"Fresh ingredients and a flavorful adventure is how I describe Tacoria.  \nThe staff is always helpful and patient.  \nI usually take out.What a disappointment! Are of the people there on downers? They all seem to have flatlined. No emotion, no affect. It's not like they could be tired from...Burritos here are sheer perfection. Stuffed with tasty meats, fresh veggies, cheese, and rice. It's bursting with flavor.  The warm toasted tortilla just..."

In [26]:
review

"Fresh ingredients and a flavorful adventure is how I describe Tacoria.  \nThe staff is always helpful and patient.  \nI usually take out.What a disappointment! Are of the people there on downers? They all seem to have flatlined. No emotion, no affect. It's not like they could be tired from...Burritos here are sheer perfection. Stuffed with tasty meats, fresh veggies, cheese, and rice. It's bursting with flavor.  The warm toasted tortilla just..."

<h1> Webscraping </h1>

In [41]:
from bs4 import BeautifulSoup
import requests
import re

In [105]:
web = str(requests.get("https://www.yelp.com/biz/tacoria-new-brunswick#reviews").content)
#replace comments with a GET THIS tag

soup = BeautifulSoup(web)
p_list = soup.find("div", {"id": "reviews"}).find_all("span", {"lang":"en"})

In [104]:
reviews = [p.get_text() for p in p_list]
reviews

['My Fave! Love it all. The food is always amazing.Delivery is always fast. Portions are huge.',
 "The wait line during rush hour, quantity, and prices do not justify the hype this place gets! Went to Tacoria with a friend for dinner around 7:45 and we waited 20 minutes just to get our order, which for me was just one taco! They seemed to prioritize delivery orders and despite it being rush hour, I was frustrated how long I had to wait for my order. Each taco is $4, which for the quality and taste wasn't worth it. However, even for a small taco, there were a lot of chicken pieces, which I appreciated! The cilantro and cotija cheese tasted delicious together. Once you go inside the restaurant, there's a beautiful backdrop, gardeny type of vibes so it's nice to sit there, talk with friends, and eat. Ordered: October 11, 2022 via Dining-In",
 "Some of the best tacos I've ever had!Tacoria is a quick walk from the hospital and a popular lunch spot so you'll likely have to wait in line aroun

In [141]:
#consider a single review, positive
r = reviews[2]
cleaned_r = (re.sub("[.!?,']", " ", r)).lower()

print(r)
print()
print(cleaned_r)

Some of the best tacos I've ever had!Tacoria is a quick walk from the hospital and a popular lunch spot so you'll likely have to wait in line around then. The line moves quickly though and your food comes out quickly as well!I tried the pulled chicken tacos which, were incredibly flavorful with lots of garlic and lime. Guac is a must add. I also appreciate that the tortilla isn't doubled and not super thick. Can't wait to go back and try more items on the menu!

some of the best tacos i ve ever had tacoria is a quick walk from the hospital and a popular lunch spot so you ll likely have to wait in line around then  the line moves quickly though and your food comes out quickly as well i tried the pulled chicken tacos which  were incredibly flavorful with lots of garlic and lime  guac is a must add  i also appreciate that the tortilla isn t doubled and not super thick  can t wait to go back and try more items on the menu 


In [145]:
from nltk.corpus import stopwords
tok = [w for w in set(nltk.word_tokenize(cleaned_r)) if w not in stopwords.words('english') and w.isalpha()]
tok

['lunch',
 'wait',
 'walk',
 'incredibly',
 'quick',
 'try',
 'guac',
 'spot',
 'add',
 'tacos',
 'thick',
 'lots',
 'back',
 'ever',
 'go',
 'tacoria',
 'items',
 'chicken',
 'must',
 'comes',
 'tried',
 'around',
 'menu',
 'also',
 'popular',
 'garlic',
 'though',
 'well',
 'tortilla',
 'doubled',
 'quickly',
 'pulled',
 'line',
 'likely',
 'appreciate',
 'moves',
 'food',
 'flavorful',
 'lime',
 'hospital',
 'super',
 'best']

In [148]:
#get nouns and adjectives
pos = nltk.pos_tag(tok)
pos


[('lunch', 'NN'),
 ('wait', 'NN'),
 ('walk', 'NN'),
 ('incredibly', 'RB'),
 ('quick', 'JJ'),
 ('try', 'NN'),
 ('guac', 'NN'),
 ('spot', 'NN'),
 ('add', 'VBP'),
 ('tacos', 'JJ'),
 ('thick', 'JJ'),
 ('lots', 'NNS'),
 ('back', 'RB'),
 ('ever', 'RB'),
 ('go', 'VBP'),
 ('tacoria', 'JJ'),
 ('items', 'NNS'),
 ('chicken', 'VBN'),
 ('must', 'MD'),
 ('comes', 'VBZ'),
 ('tried', 'JJ'),
 ('around', 'IN'),
 ('menu', 'NN'),
 ('also', 'RB'),
 ('popular', 'JJ'),
 ('garlic', 'NN'),
 ('though', 'RB'),
 ('well', 'RB'),
 ('tortilla', 'NN'),
 ('doubled', 'VBD'),
 ('quickly', 'RB'),
 ('pulled', 'VBN'),
 ('line', 'NN'),
 ('likely', 'JJ'),
 ('appreciate', 'NN'),
 ('moves', 'NNS'),
 ('food', 'NN'),
 ('flavorful', 'JJ'),
 ('lime', 'JJ'),
 ('hospital', 'NN'),
 ('super', 'NN'),
 ('best', 'JJS')]

In [153]:
nouns = [n[0] for n in pos if n[1] == 'NN' or n[1] == 'NNS']
adjs = [n[0] for n in pos if n[1] == 'JJ' or n[1] == 'JJS']

In [156]:
nouns, adjs

(['lunch',
  'wait',
  'walk',
  'try',
  'guac',
  'spot',
  'lots',
  'items',
  'menu',
  'garlic',
  'tortilla',
  'line',
  'appreciate',
  'moves',
  'food',
  'hospital',
  'super'],
 ['quick',
  'tacos',
  'thick',
  'tacoria',
  'tried',
  'popular',
  'likely',
  'flavorful',
  'lime',
  'best'])

In [179]:
#Choose some keywords to describe the place:

#3 nouns
n_samp = random.sample(nouns, 3)
#3 adjs
a_samp = random.sample(adjs, 3)
print(n_samp)
print(a_samp)

['tortilla', 'hospital', 'spot']
['best', 'likely', 'popular']


In [171]:
def spit(arr):
    out = ""
    for string in arr:
        out += string + " "
    return out

In [172]:
restuarant_name = "Tacoria"

In [173]:
import os

import openai
openai.api_key = mysecrets.key

def get_res(p):
    obj = openai.Completion.create(
            model="text-davinci-003",
            prompt=p,
            temperature=0.1,
            max_tokens=1000
        )
    
    return obj["choices"][0]["text"].replace("\n", "")


In [183]:
prompt = "Make a positive review about the restaurant " + restuarant_name + ". Use the keywords: " \
+ spit(n_samp) + spit(a_samp)


In [184]:
get_res(prompt)

"Tacoria is one of the best tortilla hospital spots I've ever been to! The food is always fresh and delicious, and the atmosphere is always lively and inviting. The staff is friendly and helpful, and the prices are very reasonable. It's no wonder why Tacoria is so popular - it's likely the best Mexican restaurant in town!"

In [10]:
prompt = "Make a positive review. Use the word: delicious"
get_res(prompt)

'I had the most delicious meal at this restaurant! The food was cooked to perfection and the flavors were amazing. I would highly recommend this place to anyone looking for a great meal.'